# Functional Programming 

## Test suite code

In [1]:
def exampleHeader[A](str: A): Unit = {
    val header = str match {
        case s: String => s"________${s}_________"
        case _ =>         s"___Invalid header____"
    }

    println(header)
}


val exampleExceptions: PartialFunction[String, Unit] = {
    case y: String => {
        exampleHeader("Exception occured")
        println(s"No match for: `${y}`")
    }
}

def headerWrap[A, B](f: PartialFunction[A, B]): A => B = {
    // a is the argument passed to the function
    a => {
        exampleHeader(a) // add the header
        f(a) // then do the function
    }
}


Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-05L8NA9:4040
SparkContext available as 'sc' (version = 2.4.8, master = local[*], app id = local-1672526062609)
SparkSession available as 'spark'


exampleHeader: [A](str: A)Unit
exampleExceptions: PartialFunction[String,Unit] = <function1>
headerWrap: [A, B](f: PartialFunction[A,B])A => B


### Template

In [ ]:
// copy this code block to create your test suite block.
val ex: PartialFunction[String, Unit] = {
    case "your test case" => {
        // your code here
        println("do something")
    }
}
val examples = headerWrap(ex)
examples("your test case") 

## Basics

### Partial Function
A `PartialFunction` in Scala is a function that is defined for a subset of its input values. A `PartialFunction` can be used to represent a function that may not be defined for all possible input values, or to define a function that has multiple cases with different behavior for different input values.

In [3]:
val ex: PartialFunction[String, Unit] = {
    case "Basic" => {
        // Basically you're making an options list here.  This function can only be called with 1, 2, or 3 
        val pf: PartialFunction[Int, String] = {
            case 1 => "one"
            case 2 => "two"
            case 3 => "three"
        }

        println(pf.isDefinedAt(1)) // true
        println(pf.isDefinedAt(4)) // false


        val pf2: PartialFunction[Int, String] = {
            case 4 => "four"
            case 5 => "five"
        }

        // You can combine two partial functions, NOTE: Using a variable here is required.
        val pf3 = pf.orElse(pf2)

        println(pf3(1)) // "one"
        println(pf3(4)) // "four"
    }


    case "Use with Try" => {
        import scala.util.{Try, Success, Failure}

        def divide(x: Int, y: Int): Int = x / y

        val result: Try[Int] = Try(divide(10, 2))

        // this is just a partial function with 3 cases
        result match {
        case Success(value) => println(value)  // prints 5
        case Failure(e: ArithmeticException) => println("Division by zero")
        case Failure(e) => e.printStackTrace()
        }
    }
}
val examples = ex.orElse(exampleExceptions)

examples("Use with Try")
examples("exampleExceptions")

// Try example



5
________Exception occured_________
No match for: `exampleExceptions`


ex: PartialFunction[String,Unit] = <function1>
examples: PartialFunction[String,Unit] = <function1>


## Function definitions

In [12]:
// definition for foldLeft
def foldLeft[B](z: B)(op: (B, A) => B): B // so it takes some element of type B, and curries a function which takes a function which takes two arguments the accumulator(B) and increment(A), (B, A)

List(1,2,3).foldLeft(0)((accumulator, x) => accumulator + x)

<console>: 25: error: not found: type A

## Higher-order functions
These are functions that take other functions as arguments or return them as output. Higher-order functions are a powerful tool for abstracting common patterns in functional programming and can help you write more concise and reusable code.

### Implement Map function
Implement a map function: You can use higher-order functions to implement a map function, which applies a given function to each element of a list and returns a new list with the transformed elements. For example:

In [4]:
def map[A, B](l: List[A], f: A=>B): List[B] = {
    l match {
        case Nil => Nil
        case h :: t => f(h) :: map(t,f)
    }
}

map: [A, B](l: List[A], f: A => B)List[B]


In [13]:
val ex: PartialFunction[String, Unit] = {
    case "convert string to int" => {
        val l: List[String] = List("1", "2", "3")
        val ints: List[Int] = l.map(_.toInt)
        println(ints)
    }
}

val examples = headerWrap(ex)

examples("convert string to int")

________convert string to int_________
List(1, 2, 3)


ex: PartialFunction[String,Unit] = <function1>
examples: String => Unit = <function1>


### Implement Fold function  
Implement a fold function: You can use higher-order functions to implement a fold function, which reduces a list to a single value by repeatedly applying a given function to the elements of the list. For example:

In [5]:

def fold[A, B](l: List[A], z: B, f: (B, A) => B): B = {
    l match {
        case Nil => z
        case h :: t => fold(t, f(z, h), f)
    }
}
val l = List(1,2,3,4,5)

val ex: PartialFunction[String, Unit] = {
    case "fold a List" => {
        // exampleHeader(str)
        val sum = fold(l, 0, (x: Int, y: Int) => x + y)
        println(sum)//15
        val product = fold(l, 1, (x: Int, y: Int) => x * y)
        println(product)//120
    }

    case "calculate length of list" => {
        // exampleHeader(str)
        def length[A](l: List[A]): Int = {
            fold(l, 0, (acc: Int, _: A) => acc + 1)
        }

        println(length(l)) // prints 5
    }

    case "reverse a list" => {
        def reverse[A](l: List[A]): List[A] = {
            fold(l, List.empty[A], (acc: List[A], x: A) => x :: acc)
        }

        println(reverse(l)) // prints List(5,4,3,2,1)
    }

    case "roman numeral to int" => {
        def romanToInt(s: String): Int = {
        val map: Map[Char, Int] = Map(
            'I' -> 1,
            'V' -> 5,
            'X' -> 10,
            'L' -> 50,
            'C' -> 100,
            'D' -> 500,
            'M' -> 1000,
        )
        s
            //convert to the list of int values
            //for example: IV => 1,5, XX => 10,10
            .map(map)
            //fold list adding negatives: if value decreases append as is, else add two more negatives
            //for example: IV => 1,5 => 1,-1,-1,5 = 4
            .foldLeft(List[Int](0))((a, b) => {
            if (a.head >= b) {
                b :: a
            } else {
                b :: -a.head :: -a.head :: a
            }
            })
            //calc sum
            .sum
        }
    }
}

 val examples = headerWrap(ex)

examples("fold a List")
examples("calculate length of list")
examples("reverse a list")



________fold a List_________
15
120
________calculate length of list_________
5
________reverse a list_________
List(5, 4, 3, 2, 1)


fold: [A, B](l: List[A], z: B, f: (B, A) => B)B
l: List[Int] = List(1, 2, 3, 4, 5)
ex: PartialFunction[String,Unit] = <function1>
examples: String => Unit = <function1>


### Custom control structures  
Create custom control structures: You can use higher-order functions to create custom control structures, such as while loops or try-catch blocks, by wrapping them in a function and passing them as arguments to other functions. For example:

In [6]:
def whileloop(cond: => Boolean)(body: => Unit): Unit = {
    if(cond){
        body
        whileloop(cond)(body)
    }
}

// catchBlock is a PartialFunction that takes a Throwable ARGUMENT and returns T the same return type as tryCatch
def tryCatch[T](body: => T)(catchBlock: PartialFunction[Throwable, T]): T = {
    try{
        body
    } catch catchBlock
}


// Example usages:
var i = 0
whileloop(i < 10){
    println(i)
    i += 1
}

def divide(x: Int, y: Int): Int = {
    if(y == 0) throw new ArithmeticException("Division by zero")
    else x/y
}

val result = tryCatch(divide(10, 0)){
    case e: ArithmeticException => 0
}
println("Result: ", result)


0
1
2
3
4
5
6
7
8
9
(Result: ,0)


whileloop: (cond: => Boolean)(body: => Unit)Unit
tryCatch: [T](body: => T)(catchBlock: PartialFunction[Throwable,T])T
i: Int = 10
divide: (x: Int, y: Int)Int
result: Int = 0


## Currying
This is a technique for converting a function that takes multiple arguments into a series of functions that each take a single argument. Currying can make it easier to partially apply functions and create specialized versions of functions.

In [8]:
val ex: PartialFunction[String, Unit] = {
    case str @ "curry booleans" => {
        // exampleHeader(str) -- leaving these here but basically these do the same as headerWrap(), if you uncomment the ex.orElse line

        // You can curry function blocks -- refer to Higher order functions whileloop and tryCatch section for more details.
        def test(x: => Boolean)(y: => Unit): Unit = {
            if(x) y
            else println("other block")
        }

        test(1 > 0){
            println("yep")
        }
        test(1 < 0){
            println("shouldn't print this")
        }

    }

    case str @ "curry function result" => {
        // exampleHeader(str)

        def takesInt(x: => Int)(y: => Unit): Unit = {
            if(x > 0) y
            else println("other block")
        }

        def sum(x: Int, y: Int): Int = x + y
        takesInt(sum(1,2)){println("greater than 0")}

    }

}

val examples = headerWrap(ex)

// val examples = ex.orElse(exampleExceptions)

examples("curry booleans")
examples("curry function result")


________curry booleans_________
yep
other block
________curry function result_________
greater than 0


ex: PartialFunction[String,Unit] = <function1>
examples: String => Unit = <function1>


## Partial applications
This is the process of creating a new function by fixing one or more arguments of an existing function. Partial application can be used to create specialized versions of functions that are easier to reuse in different contexts.

## Pure functions
These are functions that always return the same output for a given input and have no side effects. Pure functions are easier to test and reason about, and they are an important concept in functional programming.

In [ ]:

def add(x: Int, y: Int): Int = x + y // this function is pure because it doesn't modify any external variables or states(like printing to console).
def double(l : List[Int]): List[Int] = l.map(_ * 2) // doesn't modify the original list, just returns a new list.
def removeVowels(s: String): String = s.replaceAll("[aeiou]", "") // doesn't modify the original string 

## Recursion
This is a technique for defining functions that call themselves as part of their execution. Recursion is a common technique in functional programming and is often used to solve problems that can be naturally decomposed into smaller subproblems.

In [3]:
    def readIndex(x:Char) : Int = {
        x match {
            case 'I' => 1 
            case 'V' => 5
            case 'X' => 10
            case 'L' => 50
            case 'C' => 100
            case 'D' => 500
            case 'M' => 1000
            case _ => 0
        }
    }
    def romanToInt(s: String): Int = {
        def worker(l:List[Int] = s.toList.map(readIndex)):Int ={
            l match{
                case Nil => 0
                case s1::s2::ss if s1 >= s2 => s1 + worker(s2::ss) 
                case s1::s2::ss if s1 < s2 => s2 - s1 + worker(ss) 
                case s1::Nil => s1
            }
        }
        
        worker()
    }

    assert(romanToInt("IV") == 4)
    assert(romanToInt("MC") == 1100)
    assert(romanToInt("CM") == 900)

readIndex: (x: Char)Int
romanToInt: (s: String)Int


### Recursion coding challenge

There are 10 problems with increasing difficulty, try to solve each w/ Recursion.  
1. Write a recursive function that calculates the sum of all the elements in a list of integers.
2. Write a recursive function that calculates the product of all the elements in a list of integers.
3. Write a recursive function that calculates the sum of the digits of a positive integer. For example, the sum of the digits of 123 is 6.
4. Write a recursive function that calculates the factorial of a positive integer.
5. Write a recursive function that calculates the Fibonacci numbers. The Fibonacci numbers are a series of numbers where each number is the sum of the previous two numbers. The first two numbers in the series are 0 and 1, and the rest of the numbers are generated by adding the previous two numbers.
6. Write a recursive function that calculates the greatest common divisor (GCD) of two positive integers.
7. Write a recursive function that calculates the sum of the digits of all the numbers from 1 to n.
8. Write a recursive function that calculates the number of ways to make change for a given amount of money using a given set of denominations.
9. Write a recursive function that calculates the nth number in the Catalan sequence. The Catalan sequence is a series of numbers where the nth number is the sum of the binomial coefficients of the first n natural numbers.
10. Write a recursive function that calculates the number of ways to arrange a set of items in a list.

In [16]:
// Folds, map functions work great here.  

println("Recursively:")
val ex: PartialFunction[String, Unit] = {

    case "sum a list" => {
        val l = Seq(1,2,3,4,5) 
        def sum(l: Seq[Int]): Int = l match {
            case Nil => 0
            case head :: tail => head + sum(tail) // ex: first iter,   head = 1 and tail = List(2,3,4,5)
        }

        println(sum(l))
    }
    case "find product" => {
        val l = Seq(1,2,3,4,5) 
        def product(l: Seq[Int]): Int = l match {
            case Nil => 1 // can't multiply by 0
            case head :: tail => 
                println(head, tail)
                head * product(tail) 
        }

        println(product(l))
    }
    case "sum of digits in positive integer" => {

    }
}
val examples = headerWrap(ex)

examples("sum a list") 
examples("find product") 
examples("sum of digits in positive integer") 

Recursively:
________sum a list_________
15
________find product_________
(1,List(2, 3, 4, 5))
(2,List(3, 4, 5))
(3,List(4, 5))
(4,List(5))
(5,List())
120


ex: PartialFunction[String,Unit] = <function1>
examples: String => Unit = <function1>
